# Using Python for Quantum Chemistry Calculations

![](python-logo.png)

#### Python Package Python-based Simulations of Chemistry Framework (PySCF)

Resources:
https://pyscf.org/index.html
https://www.python.org/
https://avogadro.cc/

The PySCF Python package is an open source collection of electronic structure models. It is designed for easy extensibility and simple enough that it requires only a fundamental understanding. PySCF's objective is to simply and efficiently solve quantum chemistry calculations and simulate properties of given molecules or crystals. It is a highly customizable Python package and contains an array of features and extension modules for further analysis.

Main Functionalities:
- *Molecular structure
- Self-consistent field (SCF) methods
- *Density functional theory (DFT)
- Second-order Moller-Plesset perturbation theory
- GW approximation
- Configuration interaction (CISC and FCI)
- Coupled-cluster theory
- Algebraic diagrammatic construction (ADC)
- Auxiliary second-order Green's functional perturbation theory (AGF2)
- Multi-configuration self-consistent field (MCSCF)
- Time-dependent Hartree-Fock and density functional theory
- Solvation models
- *QM/MM methods
- Density fitting
- Periodic boundary conditions
- Electronic-phonon coupling
- Localized orbitals
- Seminumerical exchange (SGX)
- *Geometry optimization
*Modules utilized in following analysis

![](pyscf-logo.png)

In [4]:
# imports required to run
import pyscf
import numpy as np

OSError: dlopen(/Users/cloekwiatkowski/.conda/envs/untitled/lib/python3.10/site-packages/pyscf/lib/libnp_helper.dylib, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <09302566-E7C9-33BF-8F7D-33EDB0C5E5D3> /Users/cloekwiatkowski/.conda/envs/untitled/lib/python3.10/site-packages/pyscf/lib/libnp_helper.dylib
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

### Molecular Structure
Modules: gto

#### Initializing a Molecule

- Molecules can be created using the keyword arguments of Mole.build() or imported coordinates from a xyz file

#### Geometry

- The molecular geometry is input in Cartesian format (default unit is Angstrom)
- The units can be specified by setting the attribute unit to 'Angstrom' or 'Bohr'
- Can use the atom keyword illustrated above
- A xyz file containing coordinates of molecules can be used

In [ ]:
from pyscf import gto

In [ ]:
# Example - H2O
mol = gto.Mole()
mol.build(
    atom = '''O 0 0 0; H 0 1 0; H@2 0 0 1''', # holding coordinates for the molecular geometry
    basis = '6-31G', # specifying basis set; for individual atoms use mol.basis = {'O':'sto-3g', 'H':'6-31g', 'H@2':'cc-pvdz'}
    charge = 1, #specify charge
    spin = 1 #specify spin multiplicity
)

In [ ]:
# using sodium stearate (C18H35NaO2) coordinates from molecule construction with Avogadro software
mol = gto.M(atom = 'Sodium_Stearate_Mol.xyz')

# accessing the molecules coordinates; returns in (N,3) array format
print(mol.atom_coords(unit = 'Bohr'))

#### Point Group Symmetry
- Can invoke point group symmetry by setting attribute 'Mole.symmetry' to True
- Can assign symmetry to 'Mole.symmetry' in the initialization process
- Symmetry information held in the 'Mole' object
- Symmetry module 'symm' can detect point groups and stored in 'Mole.topgroup' and subgroup in 'Mole.groupname'

In [ ]:
# N2 molecule
mol = gto.Mole()
mol.atom = 'N 0 0 0; N 0 0 1'
mol.symmetry = True #invoking pt. group symmetry
mol.symmetry_subgroup = 'C2' #assigning subgroup symmetry
mol.build()
print(mol.topgroup) #detected point group
# Dooh
print(mol.groupname) #detected supported subgroup
# C2

In [ ]:
# O2 molecule
mol = gto.Mole()
mol.atom = 'O 0 0 0; O 0 0 1.2'
mol.spin = 2
mol.symmetry = 'D2h'
mol.build()

# Symmetry adapted orbitals are held in 'Mole.symm_orb' as a list of 2D arrays
#   each element of the list is an atomic orbital to symmetry-adapted orbital transformation matrix of an irreducible representation
#   symmetry adapted orbitals held in Mole.symm_orb
#   irreducible representations stored in Mole.irrep_name
#   internal IDs stored in Mole.irrep_id
for s, i, c in zip(mol.irrep_name, mol.irrep_id, mol.symm_orb):
    print(s, i, c.shape)

#   Ag 0 (10, 3)
#   B2g 2 (10, 1)
#   B3g 3 (10, 1)
#   B1u 5 (10, 3)
#   B2u 6 (10, 1)
#   B3u 7 (10, 1)


# calculating converged SCF energy
mf=scf.RHF(mol)
mf.kernel() # converged SCF energy = -147.631655286561

# calculating dipole moment
mf.dip_moment()

In [ ]:
# checking the occupancy of the molecular orbitals (MOs) in each irreducible representation
import numpy as np
from pyscf import symm

def myocc(mf):
    mol = mf.mol
    orbsym = symm.label_orb_symm(mol, mol.irrep_id, mol.symm_orb, mf.mo_coeff)
    doccsym = np.array(orbsym)[mf.mo_occ==2]
    soccsym = np.array(orbsym)[mf.mo_occ==1]
    for ir,irname in zip(mol.irrep_id, mol.irrep_name):
        print('%s, double-occ=%d, single-occ=%d' %
              (irname, sum(doccsym==ir), sum(soccsym==ir)))
myocc(mf)

#   Ag, double-occ = 3, single-occ = 0
#   B2g, double-occ = 0, single-occ = 1
#   B3g, double-occ = 0, single-occ = 1
#   B1u, double-occ = 2, single-occ = 0
#   B2u, double-occ = 1, single-occ = 0
#   B3u, double-occ = 1, single-occ = 0

### Density Functional Theory (DFT)
Quantum-mechanical (QM) method to calculate electronic structure of atoms, molecules, and solids
Electronic structure method

Modules: dft, pbd.dft

In [ ]:
from pyscf import dft

In [ ]:
# simple example of using the dft module with molecule HF
mol_hf = gto.M(atom = 'H 0 0 0; F 0 0 1.1', basis = 'ccpvdz', symmetry = True)
mf_hf = dft.RKS(mol_hf)
mf_hf.xc = 'lda,vwn' # default
mf_hf = mf_hf.newton() # second-order algortihm
mf_hf.kernel() #converged SCF energy = -99.7624175432877

### QM/MM Methods

The hybrid quantum mechanics/molecular mechanics as a molecular simulation methods allowing for the study of chemical processes in solution and in proteins

Combines
- ab initio QM calculations (accuracy)
- MM (speed)

Modules: qmmm

In [ ]:
from pyscf import qmmm, scf

In [ ]:
# SCF methods with MM charges

# simple example of using the qmmm module
mol = gto.M(atom = 'H 0 0 0; F 0 0 1',
            basis = 'ccpvdz')
coords = [(0.5, 0.6, 0.8)]
charges = [-0.3]
mf = qmmm.mm_charge(scf.RHF(mol), coords, charges)
mf.kernel() #converged SCF energy = -100.045455504517

### Geometry Optimization

Modules: geomopt

In [ ]:
from pyscf import geomopt
from pyscf.geomopt.geometric_solver import optimize #importing the optimize() function to invoke optimization

First way is using the optimize() function:

In [ ]:
# simple geometry optimization with N2
mol = gto.Mole(atom = 'N 0 0 0; N 0 0 1.2',
               basis = 'ccpvdz')
mf = scf.RHF(mol)

Second way is to create an optimizer() from the 'Gradient' class:

In [ ]:
mol_eq = mf.Gradients().optimizer(solver='geomeTRIC').kernel()
print(mol_eq.atom_coords())

#### Transition State Optimization
- PySCF extension qsdopt (quadratic steepest descent method)

In [ ]:
from pyscf.qsdopt.qsd_optimizer import QSD

In [ ]:
# simple example of transition state optimization with H2O molecule
mol = gto.M(atom = '''O 0 0 0; H 0 0 1.2; H 0, 0.5, -1.2''',
            basis = 'minao',
            verbose = 0,
            unit = "Bohr")
mf = scf.RHF(mol)

optimizer = QSD(mf, stationary_point = "TS")
optimizer.kernel()

#### Excited States
- For excited-state geometry optimizations, the optimized state needs to be specified in the respective 'Gradient's objects

In [ ]:
from pyscf import ci, tdscf, mcscf
from pyscf import geomopt

In [ ]:
mol = gto.Mole()
mol.atom="N; N 1, 1.1"
mol.basis= "6-31g"
mol.build()
mol1 = mol.copy()

mf = scf.RHF(mol).run()

mc = mcscf.CASCI(mf, 4,4)
mc.fcisolver.nstates = 3
excited_grad = mc.nuc_grad_method().as_scanner(state=2)
mol1 = excited_grad.optimizer().kernel()
#(or) mol1 = geomopt.optimize(excited_grad)


td = tdscf.TDHF(mf)
td.nstates = 5
excited_grad = td.nuc_grad_method().as_scanner(state=4)
mol1 = excited_grad.optimizer().kernel()
#(or) mol1 = geomopt.optimize(excited_grad)


myci = ci.CISD(mf)
myci.nstates = 2
excited_grad = myci.nuc_grad_method().as_scanner(state=1)
mol1 = excited_grad.optimizer().kernel()
#(or) geomopt.optimize(excited_grad)